<a href="https://colab.research.google.com/github/ishandahal/ml_projects/blob/main/nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
class NeuronModel():
    """Single Layered Neural Network.

    Parameters
    ------------
    num_features : int
      Number of features of the dataset
    weights : 1d-array
      Parameters for the features
    bias : 1d-array
      Parameter for the bias term

    """

    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = torch.zeros(num_features, 1, 
                                   dtype=torch.float)
        self.bias = torch.zeros(1, dtype=torch.float)
        
    def netinput_func(self, x, w, b):
        """Matrix multiply between weights and features and add the bias term.

        Parameters
        ----------
        x : {array-like}, shape = [n_examples, n_features]
          Training vectors, where n_examples is the number of examples and
          n_features is the number of features.
        w : {array-like}, shape = [num_features, 1]
          Weight parameters, where num_features is the number of features
        b : array-like, shape = [1]
          Bias parameter

        Returns
        -------
        {array-like}, shape = [n_examples, 1]
          Matrix product of weights and features plus the bias term, where
          n_examples is the number of examples 

        """

        return torch.add(torch.mm(x, w), b)

    def forward(self, x):
        """
        Forward pass through the network. netinput_func computes the matrix
        multiply followed by the activation_func which computes tanh activation

        Parameters
        ----------
        x : {array-like}, shape = [n_examples, n_features]
          Training vectors, where n_examples is the number of examples and
          n_features is the number of features.

        Returns
        -------
        {array-like}, shape = [n_examples, 1]
          Activations, where n_examples is the total number of examples. Since 
          this is a single layered network the activations are also the 
          predictions

        """

        netinputs = self.netinput_func(x, self.weights, self.bias)
        activations = self.activation_func(netinputs)
        return activations.view(-1)
        
    def backward(self, x, yhat, y):
        """
        Computes gradients (partial derivaties of loss function with respect 
        to weights and bias)

        Parameters
        ----------
        x : {array-like}, shape = [n_examples, n_features]
          Training vectors, where n_examples is the number of examples and
          n_features is the number of features.
        yhat : {array-like}, shape = [n_examples, 1]
          Activations, where n_examples are number of examples
        y : {array-like}, shape = [n_examples, 1]
          Target labels, where n_examples are number of examples

        Returns
        -------
        {array-like}, shape = [n_features, 1]
          Negative gradients of the loss with respect to the weight parameters,
          where num_features is the number of features

        {array-like}, shape = [1]
          Negative gradients of the loss with respect to the bias parameter

        """  
        
        netinputs = self.netinput_func(x, self.weights, self.bias)
        
        
        grad_loss_yhat = 2 * (yhat - y)         # x[0] = (10,)                                                      # partial derivative of the loss with respect to activation


        # grad_yhat_bias = (4 / (torch.exp(netinputs) \
        # + torch.exp(-netinputs))**2)            # dim of n                                   ## alternate method of computing the derivative.

        # grad_yhat_weights = (4 / (torch.exp(netinputs) \                                    ## alternate method of computing the derivative.  
        # + torch.exp(-netinputs))**2) * x     # dim of x

        grad_yhat_weights = (1 - self.activation_func(netinputs)**2) * x                      #dim of x (10X2)            partial derivative of activation with respect to weights 
        grad_yhat_bias = (1 - self.activation_func(netinputs)**2)                             #dim of x[0] (10,)          partial derivative of activation with respect to bias
        
        grad_loss_weights = torch.mm(grad_yhat_weights.t(), 
                                     grad_loss_yhat.view(-1, 1)) / y.size(0)                  #dim = (2X10).dot(10X1) = 2X1       Using the chain rule (inner and outer)
        grad_loss_bias = torch.sum(grad_loss_yhat * grad_yhat_bias) / y.size(0)               #dim = sum((10)*(10)) = 1          Using the chain rule (inner and outer)

        
        return (-1)*grad_loss_weights, (-1)*grad_loss_bias

    def activation_func(self, x):
        """Calculates tanh activation function"""
        return (torch.exp(x) - torch.exp(-x)) / (torch.exp(x) + torch.exp(-x))

    def loss(self, yhat, y):
        """
        Returns mean squared error
        """

        return torch.mean((yhat - y)**2)

#### Testing the class methods

In [ ]:
model = NeuronModel(num_features=2)

In [ ]:
X = torch.randn((100, 2))
y = torch.tensor([0]*50 + [1]*50)

In [ ]:
lr = 0.001

print("Running the forward and backward method for 10 rounds:")
for _ in range(10):
    y_hat = model.forward(X)
    grad_weight, grad_bias = model.backward(X, y_hat, y)
    print(f"Model weights: {model.weights[0].item():.5f}{model.weights[1].item():.5f}\n Model.bias {model.bias.item():.5f}")
    model.weights -= lr * grad_weight
    model.bias -= lr * grad_bias

print("Parameters are changing.")

Running the forward and backward method for 10 rounds:
Model weights: 0.000000.00000
 Model.bias 0.00000
Model weights: -0.00011-0.00007
 Model.bias -0.10000
Model weights: -0.00023-0.00016
 Model.bias -0.21874
Model weights: -0.00035-0.00027
 Model.bias -0.35518
Model weights: -0.00048-0.00041
 Model.bias -0.50382
Model weights: -0.00061-0.00055
 Model.bias -0.65509
Model weights: -0.00072-0.00069
 Model.bias -0.79898
Model weights: -0.00082-0.00082
 Model.bias -0.92924
Model weights: -0.00091-0.00093
 Model.bias -1.04406
Model weights: -0.00098-0.00103
 Model.bias -1.14446
Parameters are changing.
